In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
import gc
import re
import copy
import itertools
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from tqdm.notebook import tqdm
from datetime import datetime
import json,itertools
from typing import Optional
from glob import glob
import warnings
from IPython import display as ipd
warnings.filterwarnings("ignore")
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import matplotlib as mpl
from matplotlib.patches import Rectangle
import seaborn as sns
import random
from joblib import Parallel, delayed
import os, shutil
import datetime 
import holidays
import dateutil.easter as easter

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet,SGDRegressor
from sklearn.model_selection import  GroupKFold,cross_val_score

from scipy.signal import savgol_filter


from matplotlib import pyplot
import missingno as msno
import plotly.express as px

import optuna



# <div style="background-color:#fff1cc; padding: 60px;"><center>⭐Feature Engineering + 📊EDA</center>
 </div> 

# <div style="background-color:#fff1cc; padding: 20px;"><center>Import the Dataset</center>
 </div> 

In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s3e19/train.csv', parse_dates=['date'])
original_train_df = train_df.copy()
test_df = pd.read_csv('/kaggle/input/playground-series-s3e19/test.csv', parse_dates=['date'])
train_df.head()

In [ ]:
for df in [train_df, test_df]:
    df['product'] = df['product'].str.replace(' ', '_')
    df['product'] = df['product'].str.replace(':', '_')

In [ ]:
test_df.head()

# <div style="background-color:#fff1cc; padding: 20px;"><center>Explore the Dataset</center>
 </div>  

In [ ]:
print('\n  country in the train dataset\n')
print(train_df['country'].value_counts())

print('\n  country in the test dataset\n')
print(test_df['country'].value_counts())

print('\n store in the train dataset\n')
print(train_df['store'].value_counts())

print('\n store in the test dataset\n')
print(test_df['store'].value_counts())

print('\n product in the train dataset\n')
print(train_df['product'].value_counts())

print('\n product in the test dataset\n')
print(test_df['product'].value_counts())

In [ ]:
print("train min date:", train_df['date'].min())
print("train max date:", train_df['date'].max())
print("test min date:", test_df['date'].min())
print("test max date:", test_df['date'].max())

In [ ]:
plt.figure(figsize=(15,6))
train_df.groupby(['country','store','product'])['num_sold'].mean().unstack().plot(kind='bar',stacked=True)
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
train_gp = train_df.groupby('date').sum().reset_index()
plt.plot(train_gp['date'], train_gp['num_sold'])
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
for country in train_df['country'].unique():
    filt_train = train_df[train_df['country'] == country]

    train_gp = filt_train.groupby('date').sum().reset_index()
    plt.plot(train_gp['date'], train_gp['num_sold'], label=country)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
for store in train_df['store'].unique():
    filt_train = train_df[train_df['store'] == store]

    train_gp = filt_train.groupby('date').sum().reset_index()
    plt.plot(train_gp['date'], train_gp['num_sold'], label=store)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
for product in train_df['product'].unique():
    filt_train = train_df[train_df['product'] == product]

    train_gp = filt_train.groupby('date').sum().reset_index()
    plt.plot(train_gp['date'], train_gp['num_sold'], label=product)
plt.legend()
plt.show()

In [ ]:
weekly_df = train_df.groupby(["country","store", "product", pd.Grouper(key="date", freq="W")])["num_sold"].sum().rename("num_sold").reset_index()
monthly_df = train_df.groupby(["country","store", "product", pd.Grouper(key="date", freq="MS")])["num_sold"].sum().rename("num_sold").reset_index()

In [ ]:
plt.figure(figsize=(15,6))
for product in train_df['product'].unique():
    filt_train = weekly_df[weekly_df['product'] == product]

    train_gp = filt_train.groupby('date').sum().reset_index()
    plt.plot(train_gp['date'], train_gp['num_sold'], label=product)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
for product in train_df['product'].unique():
    filt_train = monthly_df[monthly_df['product'] == product]

    train_gp = filt_train.groupby('date').sum().reset_index()
    plt.plot(train_gp['date'], train_gp['num_sold'], label=product)
plt.legend()
plt.show()

In [ ]:
product_store_weights = monthly_df.groupby(["product","store"])["num_sold"].sum() / monthly_df.groupby(["product"])["num_sold"].sum()
product_store_weights

In [ ]:
store_weights = train_df.groupby("store")["num_sold"].sum()/train_df["num_sold"].sum()
store_weights

In [ ]:
new_monthly_df = monthly_df.loc[monthly_df["date"] < "2020-01-01"]
product_country_weights = new_monthly_df.groupby(["product","country"])["num_sold"].sum() / new_monthly_df.groupby(["product"])["num_sold"].sum()
product_country_weights

In [ ]:
product_df = train_df.groupby(["date","product"])["num_sold"].sum().reset_index()
product_df.head()

In [ ]:
product_ratio_df = product_df.pivot(index="date", columns="product", values="num_sold")
product_ratio_df = product_ratio_df.apply(lambda x: x/x.sum(),axis=1)
product_ratio_df = product_ratio_df.stack().rename("ratios").reset_index()
product_ratio_df.head(4)

In [ ]:
product_ratio_df.tail()

In [ ]:
f,ax = plt.subplots(figsize=(20,10))
sns.lineplot(data = product_ratio_df, x="date", y="ratios", hue="product");

# <div style="background-color:#fff1cc; padding: 20px;"><center> Feature Engineering</center>
 </div>  

In [ ]:
train_df = train_df.groupby(["date"])["num_sold"].sum().reset_index()

In [ ]:
weekly_df = train_df.groupby([pd.Grouper(key="date", freq="W")])["num_sold"].sum().rename("num_sold").reset_index()
monthly_df = train_df.groupby([pd.Grouper(key="date", freq="MS")])["num_sold"].sum().rename("num_sold").reset_index()

In [ ]:
train_nocovid_df = train_df.loc[~((train_df["date"] >= "2020-03-01") & (train_df["date"] < "2020-06-01"))]
f,ax = plt.subplots(figsize=(20,10))
sns.lineplot(data = train_nocovid_df, x="date", y="num_sold");

In [ ]:
train_df = train_nocovid_df

#get the dates to forecast for
test_all_df = test_df.groupby(["date"])["id"].first().reset_index().drop(columns="id")
#keep dates for later
test_all_df_dates = test_all_df[["date"]]

In [ ]:
def feature_engineer(df):
    new_df = df.copy()
    new_df["month"] = df["date"].dt.month
    new_df["month_sin"] = np.sin(new_df['month'] * np.pi / 24)
    new_df["month_cos"] = np.cos(new_df['month'] * np.pi / 24)
    
    new_df["day"] = df["date"].dt.day
    #new_df["day_sin"] = np.sin(new_df['day'] * (2 * np.pi / 12))
    new_df["day_sin"] = np.sin(new_df['day'] * np.pi / 62)
    
    new_df["day_of_week"] = df["date"].dt.dayofweek
    new_df["day_of_week"] = new_df["day_of_week"].apply(lambda x: 0 if x<=3 else(1 if x==4 else (2 if x==5 else (3))))
    
    new_df['friday'] = new_df.date.dt.weekday.eq(4).astype(np.uint8)
    new_df['saturday'] = new_df.date.dt.weekday.eq(5).astype(np.uint8)
    new_df['sunday'] = new_df.date.dt.weekday.eq(6).astype(np.uint8)
    
    new_df["day_of_year"] = df["date"].dt.dayofyear

    new_df["day_of_year"] = new_df.apply(lambda x: x["day_of_year"]-1 if (x["date"] > pd.Timestamp("2020-02-29") and x["date"] < pd.Timestamp("2021-01-01"))  else x["day_of_year"], axis=1)
    
    important_dates = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,16,17, 124, 125, 126, 127, 140, 141,142, 167, 168, 169, 170, 171, 173, 174, 175, 176, 177, 178, 179,
                  180, 181, 203, 230, 231, 232, 233, 234, 282, 289, 290, 307, 308, 309, 310, 311, 312, 313, 317, 318, 319, 320, 360, 361, 362, 363, 364, 365]
    

    new_df["important_dates"] = new_df["day_of_year"].apply(lambda x: x if x in important_dates else 0)
    
    new_df["year"] = df["date"].dt.year - 2016
    
    easter_date = new_df.date.apply(lambda date: pd.Timestamp(easter.easter(date.year)))

    for day in list(range(-5, 5)) + list(range(40, 48)):
        new_df[f'easter_{day}'] = (new_df.date - easter_date).dt.days.eq(day)
        
    for col in new_df.columns :
        if 'easter' in col :
            new_df = pd.get_dummies(new_df, columns = [col], drop_first=True)
            
    for day in range(24, 32):
        new_df[f'Dec_{day}'] = new_df.date.dt.day.eq(day) & new_df.date.dt.month.eq(12)
        
        
    new_df = new_df.drop(columns=["date","month","day", "day_of_year"])
    new_df = pd.get_dummies(new_df, columns = ["important_dates","day_of_week"], drop_first=True)
    
    return new_df

In [ ]:
def get_holidays(df):
    years_list = [2017, 2018, 2019, 2020, 2021]

    holiday_BE = holidays.CountryHoliday('BE', years = years_list)
    holiday_FR = holidays.CountryHoliday('FR', years = years_list)
    holiday_DE = holidays.CountryHoliday('DE', years = years_list)
    holiday_IT = holidays.CountryHoliday('IT', years = years_list)
    holiday_PL = holidays.CountryHoliday('PL', years = years_list)
    holiday_ES = holidays.CountryHoliday('ES', years = years_list)

    holiday_dict = holiday_BE.copy()
    holiday_dict.update(holiday_FR)
    holiday_dict.update(holiday_DE)
    holiday_dict.update(holiday_IT)
    holiday_dict.update(holiday_PL)
    holiday_dict.update(holiday_ES)

    df['holiday_name'] = df['date'].map(holiday_dict)
    df['is_holiday'] = np.where(df['holiday_name'].notnull(), 1, 0)
    df['holiday_name'] = df['holiday_name'].fillna('Not Holiday')
    
    return df

In [ ]:
train_all_df = get_holidays(train_df)
test_all_df = get_holidays(test_all_df)

In [ ]:
train_all_df = feature_engineer(train_all_df)
test_all_df = feature_engineer(test_all_df)

In [ ]:
train_all_df['num_sold'] = np.log(train_all_df['num_sold'])

In [ ]:
display(train_all_df.head(2))
display(test_all_df.head(2))